In [ ]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.envs.registration import register
from gymnasium.utils.env_checker import check_env

from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv

SNAKE.ipynb
SNAKE - head.ipynb
SNAKE - smaller_view.ipynb

# Maak de custom gymnasium omgeving

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import random

import pygame
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import random

class SnakeEnv(gym.Env):
    metadata = {"render_modes": ["human"]}

    def __init__(self, render_mode=None):
        super(SnakeEnv, self).__init__()
        self.grid_size = 20
        self.cell_size = 30
        self.action_space = spaces.Discrete(4)  # 0=UP, 1=RIGHT, 2=DOWN, 3=LEFT
        self.observation_space = spaces.Box(
            low=0, high=2,
            shape=(3, self.grid_size, self.grid_size),
            dtype=np.int32
)
        self.render_mode = render_mode
        self.window = None
        self.clock = None
        self.obs_buffer = []    

    def reset(self, seed=None, options=None):
        self.snake = [(5, 5), (5, 5), (5, 5)]
        self.direction = 1
        self.food = self._place_food()
        self.done = False
        if self.render_mode == "human":
            self._init_render()
        return self._get_obs(), {}

    def _place_food(self):
        while True:
            food = (random.randint(0, 9), random.randint(0, 9))
            if food not in self.snake:
                return food

    def _get_obs(self):
        obs = np.zeros((self.grid_size, self.grid_size), dtype=np.int32)
        for x, y in self.snake:
            obs[y][x] = 1
        
        head_x, head_y = self.snake[0]
        obs[head_y][head_x] = 3  # 👈 Mark head as 3
        
        fx, fy = self.food
        obs[fy][fx] = 2
    
        self.obs_buffer.append(obs)
        if len(self.obs_buffer) > 3:
            self.obs_buffer.pop(0)
    
        while len(self.obs_buffer) < 3:
            self.obs_buffer.insert(0, np.zeros((self.grid_size, self.grid_size), dtype=np.int32))
    
        return np.array(self.obs_buffer)

    def step(self, action):
        if self.done:
            return self._get_obs(), 0, True, False, {}

        if abs(action - self.direction) == 2:
            action = self.direction

        self.direction = action

        dx = [0, 1, 0, -1]
        dy = [-1, 0, 1, 0]
        head_x, head_y = self.snake[0]
        new_head = (head_x + dx[action], head_y + dy[action])

        if (new_head in self.snake or
            not 0 <= new_head[0] < self.grid_size or
            not 0 <= new_head[1] < self.grid_size):
            self.done = True
            return self._get_obs(), -10, True, False, {}

        self.snake.insert(0, new_head)

        fx, fy = self.food
        old_dist = abs(head_x - fx) + abs(head_y - fy)
        new_dist = abs(new_head[0] - fx) + abs(new_head[1] - fy)

        self.steps = 0
        self.steps += 1

        if new_head == self.food:
            reward = 50
            self.food = self._place_food()
            self.steps = 0  
        else:
            reward = (old_dist - new_dist) * 0.5 - 0.2
            self.snake.pop()

        if self.steps >= 100:
            reward= -10
            self.done = True
            return self._get_obs(), reward, True, False, {}


        if self.render_mode == "human":
            self.render()
            

        return self._get_obs(), reward, False, False, {}

    def _init_render(self):
        pygame.init()
        self.window = pygame.display.set_mode(
            (self.grid_size * self.cell_size, self.grid_size * self.cell_size))
        pygame.display.set_caption("Snake AI")
        self.clock = pygame.time.Clock()

    def render(self):
        if self.window is None:
            self._init_render()

        self.window.fill((0, 0, 0))
        for x, y in self.snake:
            pygame.draw.rect(
                self.window,
                (0, 255, 0),
                pygame.Rect(x * self.cell_size, y * self.cell_size, self.cell_size, self.cell_size)
            )

        fx, fy = self.food
        pygame.draw.rect(
            self.window,
            (255, 0, 0),
            pygame.Rect(fx * self.cell_size, fy * self.cell_size, self.cell_size, self.cell_size)
        )

        pygame.display.flip()
        self.clock.tick(10)

    def close(self):
        if self.window:
            pygame.quit()


In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv
import gymnasium as gym

from stable_baselines3.common.callbacks import BaseCallback

class EvalAndRenderCallback(BaseCallback):
    def __init__(self, eval_env, render_freq=10_000, verbose=0):
        super().__init__(verbose)
        self.eval_env = eval_env
        self.render_freq = render_freq
        self.episodes_run = 0

    def _on_step(self) -> bool:
        if self.num_timesteps % self.render_freq == 0:
            obs, _ = self.eval_env.reset()
            done = False
            while not done:
                action, _ = self.model.predict(obs, deterministic=True)
                obs, reward, done, truncated, info = self.eval_env.step(action)
                self.eval_env.render()
        return True


# Maak een instance van je custom omgeving
env = SnakeEnv(render_mode=None)

# Check of alles compatibel is
#check_env(env)

# Wrap de omgeving (nodig voor Stable-Baselines3)
vec_env = DummyVecEnv([lambda: SnakeEnv(render_mode=None)])


model = PPO(
    "MlpPolicy",             
    vec_env,
    verbose=1,
    learning_rate=2.5e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    tensorboard_log="./ppo_snake_tensorboard/",
    device="cpu"
)

eval_env = SnakeEnv(render_mode="human")
callback = EvalAndRenderCallback(eval_env=eval_env, render_freq=1000)


model.learn(total_timesteps=100_000, callback=None)

model.save("ppo_snake_timeframe.5lr")
# Om later opnieuw te laden:
# model = PPO.load("ppo_snake")

test_env = SnakeEnv(render_mode="human")
obs, _ = test_env.reset()
done = False

while not done:
    action, _states = model.predict(obs)
    obs, reward, done, truncated, info = test_env.step(action)
    test_env.render()

test_env.close()


NameError: name 'SnakeEnv' is not defined

In [6]:
from stable_baselines3 import PPO

# Recreate the environment (no training wrapper needed)
eval_env = SnakeEnv(render_mode="human")  # enable rendering

# Load your trained model
model = PPO.load("ppo_snake_timeframe_10M", env=eval_env,device='cpu')


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [2]:
import time
import numpy as np
from stable_baselines3 import PPO

# Recreate the environment (no training wrapper needed)
eval_env = SnakeEnv(render_mode="human")  # enable rendering

# Load your trained model
model_path = "ppo_snake_timeframe_10M.zip"
model = PPO.load(model_path, env=eval_env,device='cpu')

num_episodes = 10
total_rewards = []
food_counts = []

highscore = 0

for episode in range(num_episodes):
    obs, _ = eval_env.reset()
    done = False
    episode_reward = 0
    step_count = 0
    food_eaten = 0
    steps_since_food = 0  


    while not done:
        action, _ = model.predict(obs, deterministic=True)
        prev_snake_len = len(eval_env.snake)

        obs, reward, done, truncated, info = eval_env.step(action)
        episode_reward += reward
        step_count += 1
        steps_since_food += 1  # tel stappen sinds laatste appel

        # Check of voedsel is gegeten
        if len(eval_env.snake) > prev_snake_len:
            food_eaten += 1
            steps_since_food = 0  # reset bij eten

        # Stop als er 200 stappen geen voedsel is gegeten
        if steps_since_food >= 200:
            print("⚠️  Terminating: no food eaten in 200 steps.")
            break

        time.sleep(0.1)


    total_rewards.append(episode_reward)
    food_counts.append(food_eaten)
    highscore = max(highscore, food_eaten)

    print(f"\n✅ Episode {episode+1} finished.")
    print(f"🔸 Reward: {episode_reward:.2f}")
    print(f"🍎 Food eaten: {food_eaten}")
    print(f"🏆 Highscore so far: {highscore}")

# Na alle episodes
avg_reward = sum(total_rewards) / len(total_rewards)
avg_food = sum(food_counts) / len(food_counts)

print("\n==== Test Summary ====")
print(f"Average reward: {avg_reward:.2f}")
print(f"Average food per episode: {avg_food:.2f}")
print(f"Highscore (most food): {highscore}")


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\Jaspe\.conda\envs\Pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



✅ Episode 1 finished.
🔸 Reward: 41.20
🍎 Food eaten: 1
🏆 Highscore so far: 1

✅ Episode 2 finished.
🔸 Reward: 139.80
🍎 Food eaten: 3
🏆 Highscore so far: 3

✅ Episode 3 finished.
🔸 Reward: 90.10
🍎 Food eaten: 2
🏆 Highscore so far: 3

✅ Episode 4 finished.
🔸 Reward: 292.10
🍎 Food eaten: 6
🏆 Highscore so far: 6

✅ Episode 5 finished.
🔸 Reward: 138.50
🍎 Food eaten: 3
🏆 Highscore so far: 6

✅ Episode 6 finished.
🔸 Reward: 86.80
🍎 Food eaten: 2
🏆 Highscore so far: 6

✅ Episode 7 finished.
🔸 Reward: 188.10
🍎 Food eaten: 4
🏆 Highscore so far: 6

✅ Episode 8 finished.
🔸 Reward: -8.20
🍎 Food eaten: 0
🏆 Highscore so far: 6

✅ Episode 9 finished.
🔸 Reward: 339.00
🍎 Food eaten: 7
🏆 Highscore so far: 7

✅ Episode 10 finished.
🔸 Reward: 136.60
🍎 Food eaten: 3
🏆 Highscore so far: 7

==== Test Summary ====
Average reward: 144.40
Average food per episode: 3.10
Highscore (most food): 7
